## Part 1: Preprocessing

In [3]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [4]:
# Determine the number of unique values in each column.
attrition_df.nunique()

Age                         43
Attrition                    2
BusinessTravel               3
Department                   3
DistanceFromHome            29
Education                    5
EducationField               6
EnvironmentSatisfaction      4
HourlyRate                  71
JobInvolvement               4
JobLevel                     5
JobRole                      9
JobSatisfaction              4
MaritalStatus                3
NumCompaniesWorked          10
OverTime                     2
PercentSalaryHike           15
PerformanceRating            2
RelationshipSatisfaction     4
StockOptionLevel             4
TotalWorkingYears           40
TrainingTimesLastYear        7
WorkLifeBalance              4
YearsAtCompany              37
YearsInCurrentRole          19
YearsSinceLastPromotion     16
YearsWithCurrManager        18
dtype: int64

In [5]:
# Create y_df with the Attrition and Department columns
y_df = attrition_df[['Attrition', 'Department']].copy()


In [6]:
# Create a list of at least 10 column names to use as X data
X_columns = [
    'Age', 
    'DistanceFromHome', 
    'Education', 
    'JobLevel', 
    'WorkLifeBalance', 
    'NumCompaniesWorked', 
    'PercentSalaryHike', 
    'TotalWorkingYears', 
    'TrainingTimesLastYear', 
    'YearsAtCompany'
]


# Create X_df using your selected columns
X_df = attrition_df[X_columns].copy()

# Show the data types for X_df
print(X_df.dtypes)



Age                      int64
DistanceFromHome         int64
Education                int64
JobLevel                 int64
WorkLifeBalance          int64
NumCompaniesWorked       int64
PercentSalaryHike        int64
TotalWorkingYears        int64
TrainingTimesLastYear    int64
YearsAtCompany           int64
dtype: object


In [7]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, test_size=0.2, random_state=42)


In [8]:
# Convert your X data to numeric data types however you see fit
# Add new code cells as necessary
print(X_df.dtypes)


Age                      int64
DistanceFromHome         int64
Education                int64
JobLevel                 int64
WorkLifeBalance          int64
NumCompaniesWorked       int64
PercentSalaryHike        int64
TotalWorkingYears        int64
TrainingTimesLastYear    int64
YearsAtCompany           int64
dtype: object


In [9]:
# Create a StandardScaler
scaler = StandardScaler()

# Fit the StandardScaler to the training data
scaler.fit(X_train)

# Scale the training and testing data
# Transform the training data
X_train_scaled = scaler.transform(X_train)

# Transform the testing data with the same scaler
X_test_scaled = scaler.transform(X_test)


In [10]:
# Create a OneHotEncoder for the Department column
from sklearn.preprocessing import OneHotEncoder

department_encoder = OneHotEncoder(sparse_output=False)

# Fit the encoder to the training data
department_encoder.fit(y_train[['Department']])


# Create two new variables by applying the encoder
# to the training and testing data
y_train_dept = department_encoder.transform(y_train[['Department']])
y_test_dept = department_encoder.transform(y_test[['Department']])

print(y_train_dept)

[[0. 1. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 ...
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]


In [11]:
# Create a OneHotEncoder for the Attrition column
att_encoder = OneHotEncoder(sparse_output=False)

# Fit the encoder to the training data
att_encoder.fit(y_train[['Attrition']])


# Create two new variables by applying the encoder
# to the training and testing data
y_train_att = att_encoder.transform(y_train[['Attrition']])
y_test_att = att_encoder.transform(y_test[['Attrition']])

print(y_train_att)

[[1. 0.]
 [1. 0.]
 [1. 0.]
 ...
 [0. 1.]
 [1. 0.]
 [1. 0.]]


## Create, Compile, and Train the Model

In [13]:
# Find the number of columns in the X training data
num_features = X_train.shape[1]


# Create the input layer
input_layer = layers.Input(shape=(X_train.shape[1],), name='input_features')

# Create at least two shared layers
shared_layer1 = layers.Dense(64, activation='relu', name='shared1')(input_layer)
shared_layer2 = layers.Dense(128, activation='relu', name='shared2')(shared_layer1)

In [14]:
# Create a branch for Department
# with a hidden layer and an output layer

# Create the hidden layer
department_hidden = layers.Dense(32, activation='relu', name='department_hidden')(shared_layer2)


# Create the output layer
department_output = layers.Dense(3, activation='softmax', name='department_output')(department_hidden)



In [20]:
# Create a branch for Attrition
# with a hidden layer and an output layer

# Create the hidden layer
attrition_hidden = layers.Dense(32, activation='relu', name='attrition_hidden')(shared_layer2)


# Create the output layer
attrition_output = layers.Dense(2, activation='sigmoid', name='attrition_output')(attrition_hidden)



In [22]:
# Create the model
model = Model(inputs=input_layer, outputs={'department_output' : department_output, 'attrition_output' : attrition_output})


# Compile the model
model.compile(optimizer='adam',
              loss={'department_output': 'categorical_crossentropy', 'attrition_output': 'binary_crossentropy'},
              metrics={'department_output': 'accuracy', 'attrition_output': 'accuracy'})

# Summarize the model
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_features (InputLayer)   │ (None, 10)                │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ shared1 (Dense)               │ (None, 64)                │             704 │ input_features[0][0]       │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ shared2 (Dense)               │ (None, 128)               │           8,320 │ shared1[0][0]              │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ attrition_hidden (Dense)      │ (None, 32)                │           4,128 │ shared2[0][0]              │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ department_hidden (Dense)     │ (None, 32)                │           4,128 │ shared2[0][0]              │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ attrition_output (Dense)      │ (None, 2)                 │              66 │ attrition_hidden[0][0]     │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ department_output (Dense)     │ (None, 3)                 │              99 │ department_hidden[0][0]    │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 17,445 (68.14 KB)

 Trainable params: 17,445 (68.14 KB)

 Non-trainable params: 0 (0.00 B)

In [24]:
print("X_train_scaled shape:", X_train_scaled.shape)
print("y_train_dept shape:", y_train_dept.shape)
print("y_train_att shape:", y_train_att.shape)

X_train_scaled shape: (1176, 10)
y_train_dept shape: (1176, 3)
y_train_att shape: (1176, 2)


In [34]:
# Train the model
model.fit(
    X_train_scaled,
    {
        'department_output': y_train_dept, 'attrition_output': y_train_att
    },
    epochs=100,
    batch_size=32,
    validation_split=0.2
)


Epoch 1/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - attrition_output_accuracy: 0.9070 - attrition_output_loss: 0.2235 - department_output_accuracy: 0.8955 - department_output_loss: 0.3335 - loss: 0.5569 - val_attrition_output_accuracy: 0.7458 - val_attrition_output_loss: 0.5832 - val_department_output_accuracy: 0.6568 - val_department_output_loss: 1.0324 - val_loss: 1.6747
Epoch 2/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - attrition_output_accuracy: 0.9162 - attrition_output_loss: 0.2080 - department_output_accuracy: 0.8795 - department_output_loss: 0.3396 - loss: 0.5470 - val_attrition_output_accuracy: 0.7585 - val_attrition_output_loss: 0.6333 - val_department_output_accuracy: 0.6398 - val_department_output_loss: 1.0003 - val_loss: 1.6939
Epoch 3/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - attrition_output_accuracy: 0.9093 - attrition_output_loss: 0.2154 - department_output_accuracy: 0.8917 - department_output_loss: 0.2917 - loss: 0.5066 - val_attrition_output_accuracy: 0.7542 - 

In [32]:
# Evaluate the model with the testing data
test_results = model.evaluate(X_test_scaled, {'department_output': y_test_dept, 'attrition_output': y_test_att})
test_results

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - attrition_output_accuracy: 0.7911 - attrition_output_loss: 0.5467 - department_output_accuracy: 0.5988 - department_output_loss: 1.0556 - loss: 1.6072


[1.4740127325057983,
 0.47857600450515747,
 0.9745113253593445,
 0.8061224222183228,
 0.6326530575752258]

In [44]:
# Print the accuracy for both department and attrition
print(f"Department Accuracy: {test_results[1]}")
print(f"Attrition Accuracy: {test_results[3]}")

Department Accuracy: 0.47857600450515747
Attrition Accuracy: 0.8061224222183228


# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not? 

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1. Acurracy is not the best metric for this data. for department accuracy, with three categories, imbalance might not be as severe unless one department overwhelmingly dominates. Here, accuracy might still be somewhat informative, but check for class distribution. For attrition, if attrition is a rare event, high accuracy might be achieved by always predicting "no attrition," which isn't useful for business decisions.
2. I used softmax for the Department output because it had 3 catagories (multi-class) and therefor was not binary. And I used Sigmoid for the attrition output because it was binary.
3. Not sure it will always improve, but we can always expiriemnt with different layers, neurons per layer, or different types of layers. We could also try different ectivation functions that we havent covered in class, like LeakyReLU, ELU, or SELU. for the output layers, we should make sure the activation function matches the task.